# Demo for paper "First Order Motion Model for Image Animation"

In [2]:
# Mount Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/My\ Drive/Capstone/deepfakes

/content/gdrive/My Drive/Capstone/deepfakes


In [5]:
# Run Once
!pip install PyYAML==5.3.1
!git clone https://github.com/AliaksandrSiarohin/first-order-model
%cd first-order-model

     |████████████████████████████████| 276kB 5.4MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=fdde2d1cfeed4d2d105af8c515c1b6c2ecc52ff4eb7afcf319f8cf0293a94a11
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'first-order-model'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 249 (delta 0), reused 0 (delta 0), pack-reused 246
Receiving objects: 100% (249/249), 72.13 MiB | 16.49 MiB/s, done.
Resolving deltas: 100% (121/121), done.
Checking out files: 100% (46/46), done.
/content/gdrive/My Drive/Capstone/deepfakes/first-order-model


In [4]:
%cd first-order-model

/content/gdrive/My Drive/Capstone/deepfakes/first-order-model


In [37]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte
warnings.filterwarnings("ignore")

In [43]:
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=60, repeat_delay=1000)
    plt.close()
    return ani


def deep_fake(image, video, i):
  source_image = imageio.imread(f'/content/gdrive/My Drive/Capstone/deepfakes/media/{image}')
  driving_video = imageio.mimread(f'/content/gdrive/My Drive/Capstone/deepfakes/media/{video}', memtest=False) # Needed if video quality is to good.
  
  #Resize image and video to 256x256
  source_image = resize(source_image, (256, 256))[..., :3]
  driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

  generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/Capstone/deepfakes/media/vox-cpk.pth.tar') 

  predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

  #save resulting video
  imageio.mimsave(f'../generated{i}.gif', [img_as_ubyte(frame) for frame in predictions])
  #video can be downloaded from /content folder

  return HTML(display(source_image, driving_video, predictions).to_html5_video())

In [44]:
for i in range(1, 2):
  deep_fake(f"seed{i}.png", "04.mp4", i)

100%|██████████| 211/211 [00:04<00:00, 48.96it/s]
